In [1]:
import sys
sys.path.append("..")
from SINC_functions import *
import matplotlib.pyplot as plt

In [2]:
x = np.genfromtxt("OTU.csv")
m = np.genfromtxt("Cytokine.csv")

In [ ]:
v0 = 0.01
v1 = 10
lamb = 150
vB = 1
a_gamma = 2
b_gamma = 2
a_pi = 2
b_pi = 2
max_iters = 40
tol_prec = 0.01
tol_elbo = 1
tol_B = 0.01
cpus = 1
omega, EZ, phi,B,iters_total, elbo, elbo_score  = SINC(x, m, v0, v1, lamb, vB,a_gamma,b_gamma,a_pi,b_pi, max_iters, tol_prec, tol_elbo, cpus)

('ELBO:', nan, 'CHANGE IN ELBO:', 10000)


In [22]:
np.savetxt("Estimated_Precision.csv",omega, delimiter=",")
np.savetxt("Estimated_MicroMicro_Edge_Prob.csv",EZ, delimiter=",")
np.savetxt("Estimated_Coefficients.csv",B, delimiter=",")
np.savetxt("Estimated_CytoMicro_Edge_Prob.csv",phi, delimiter=",")

In [24]:
v0 = 0.01
v1 = 10
lamb = 150
vB = 1
a_gamma = 2
b_gamma = 2
a_pi = 2
b_pi = 2
max_iters = 50
tol_prec = 0.01
tol_elbo = 1
tol_B = 0.01
cpus = 1
omega, EZ, phi,B,iters_total, elbo, elbo_score  = SINC_fixed_Omega(x, m, v0, v1, lamb, vB,a_gamma,b_gamma,a_pi,b_pi, max_iters, tol_prec, tol_elbo, cpus)

(90, 3, 29)
(-31910.404914961357, -9526.920829688515, -44351.32463209643, -85788.65037674631, 85788.65037674631)
('Finished Iteration 1: ', 'change in Omega,B,Z', 0.0, 0.9557814934402833, 4.851179829522034)
(90, 3, 29)
(-16895.163287177405, -9526.920829688515, -44344.532495986125, -70766.61661285204, 15022.033763894273)
('Finished Iteration 2: ', 'change in Omega,B,Z', 0.0, 0.539547135969085, 1.237017300881842)
(90, 3, 29)
(-14367.928232024504, -9526.920829688515, -44340.15661734701, -68235.00567906004, 2531.610933792006)
('Finished Iteration 3: ', 'change in Omega,B,Z', 0.0, 0.05226030986781538, 0.5203641675067794)
(90, 3, 29)
(-13523.518510667855, -9526.920829688515, -44335.96416991076, -67386.40351026713, 848.6021687929024)
('Finished Iteration 4: ', 'change in Omega,B,Z', 0.0, 0.04577900478294184, 0.6237823313444393)
(90, 3, 29)
(-13344.949335265945, -9526.920829688515, -44331.50722054069, -67203.37738549514, 183.02612477199)
('Finished Iteration 5: ', 'change in Omega,B,Z', 0.0, 0

In [25]:
np.savetxt("Estimated_Precision_fixed_Omega.csv",omega, delimiter=",")
np.savetxt("Estimated_MicroMicro_Edge_Prob_fixed_Omega.csv",EZ, delimiter=",")
np.savetxt("Estimated_Coefficients_fixed_Omega.csv",B, delimiter=",")
np.savetxt("Estimated_CytoMicro_Edge_Prob_fixed_Omega.csv",phi, delimiter=",")

In [28]:
v0 = 0.01
v1 = 10
lamb = 150
vB = 1
a_gamma = 2
b_gamma = 2
a_pi = 2
b_pi = 2
max_iters = 50
tol_prec = 0.01
tol_elbo = 1
tol_B = 0.01
cpus = 1
omega, EZ, phi,B,iters_total, elbo, elbo_score  = SINC_fixed_B(x, m, v0, v1, lamb, vB,a_gamma,b_gamma,a_pi,b_pi, max_iters, tol_prec, tol_elbo, cpus)

(0, -19284.310187152965, -44193.72016420862, -63478.03035136159, 63478.03035136159)
('Finished Iteration 1: ', 'change in Omega,B,Z', 66.83899565941304, 0.0, 3.9914072154870532)
(0, -11801.198737947909, -44188.66999765817, -55989.868735606084, 7488.161615755504)
('Finished Iteration 2: ', 'change in Omega,B,Z', 0.12390842210407205, 0.0, 1.0932315549565677)
(0, -11251.000765361678, -44187.534859449144, -55438.53562481082, 551.3331107952617)
('Finished Iteration 3: ', 'change in Omega,B,Z', 0.1332889533125058, 0.0, 0.752286427347908)
(0, -11178.70869963215, -44186.947844746974, -55365.656544379126, 72.87908043169591)
('Finished Iteration 4: ', 'change in Omega,B,Z', 0.07899243449493643, 0.0, 0.6685294686990195)
(0, -11169.411545477271, -44186.49077901219, -55355.902324489456, 9.754219889669912)
('Finished Iteration 5: ', 'change in Omega,B,Z', 0.042276583615579066, 0.0, 0.5527262258626018)
(0, -11166.472768730942, -44186.133711957445, -55352.60648068839, 3.2958438010682585)
('Finished It

In [29]:
np.savetxt("Estimated_Precision_fixed_B.csv",omega, delimiter=",")
np.savetxt("Estimated_MicroMicro_Edge_Prob_fixed_B.csv",EZ, delimiter=",")
np.savetxt("Estimated_Coefficients_fixed_B.csv",B, delimiter=",")
np.savetxt("Estimated_CytoMicro_Edge_Prob_fixed_B.csv",phi, delimiter=",")